## Passei Direto Technical challenge - 2

#### Initialization of spark context

In [1]:
!pip install python-slugify

In [2]:
from slugify import slugify
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Create Spark session
spark = SparkSession.builder \
    .appName('Passei Direto - Challenge 2') \
    .master('local[*]') \
    .getOrCreate()

#### Aggregating all BASE B files into one dataframe and renaming columns

In [5]:
df = spark.read.json(['BASE B/*.json'])

# renaming columns
df = df.toDF(*(slugify(c).replace('-', '_') for c in df.columns))
df.count()

698942

In [6]:
df.limit(10).toPandas().head(10)

,last_accessed_url,page_category,page_category_1,page_category_2,page_category_3,page_name,at,browser,carrier,city_name,...,name,nth,os_ver,platform,region,session_uuid,studentid_clienttype,type,user_type,uuid
0,/,perfil,perfil,Undefined,Undefined,/perfil/22482764/materiais,2017-11-16 02:10:20,Chrome 62,Telemar Norte Leste S.a.,None,...,Page View,17,,Linux,None,188031bec37fc43b737c2c49349076700ae89128,34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364...,e,known,1b3ed1360694ceae79f6361ed11b03cf245311c8
1,/disciplina/estudos-dirigidos-viii?type=6&mate...,busca,busca,Undefined,Undefined,/busca?disciplina=686342&disciplina=669865&dis...,2017-11-16 02:10:26,Firefox 56,Telefonica Brasil S.a.,None,...,Page View,19,,Windows 7,None,e3fe75ffce700fef07fd93e66312117ba008f046,23fa8b361cc7ea61dea852d72eba9129f05981077e1f65...,e,known,b0110b5beac39737f52678ffdba6c565bb72d8e2
2,/ranking/curso,disciplina,disciplina,marketing-pessoal,Undefined,/disciplina/marketing-pessoal,2017-11-16 02:10:22,Chrome 61,Tim Celular S.a.,None,...,Page View,2,,Windows 7,None,e0841700c23ea9aaadf1104690e7d6e12a8a771a,14cc805c72b0eec6292916da59c9c45a8aa621a1138eb6...,e,known,bea05ad3a2fc2b621bcdb048eafa4e70f5344b17
3,None,home,None,None,None,/,2017-11-16 02:10:34,Mobile Safari 10,Internexa Brasil Operadora De Telecomunicacoes...,None,...,Page View,2,10.3.2,iOS,None,318775acadbb0c72430ac2eeb05782589fbd1cf7,98774aeb803eb08e08020309fe8472af79033fbae26ea4...,e,known,a8ee07ca847a3126007ee0f5ceec8d6fa0d92727
4,None,disciplina,disciplina,cinesiologia,Undefined,/disciplina/cinesiologia?type=6&materialid=175...,2017-11-16 02:10:53,Chrome 61,Hipernet Servico De Comunicacao Ltda Me,None,...,Page View,11,,Windows 8.1,None,91bf1fbddb681569c914a8bb05fa0e260d9d92bd,122ac90aee4fc73660620456712cfea98d7bd720a1fc71...,e,known,ad6d53dc93ec8d8c30050f4391c9ecdf7da13710
5,None,disciplina,disciplina,direito-penal-i,Undefined,/disciplina/direito-penal-i?type=6&materialid=...,2017-11-16 02:10:44,Chrome 58,Telefonica Brasil S.a.,None,...,Page View,6,,Windows 7,None,d52b6c4881b52297521463e789d6721cb85e08b6,d799aecbb0a87cdd5412c68172189cc65d84644ac26e34...,e,known,23886a3fe7cb5853b3aeb7f3690c42ae1b31d7f3
6,None,disciplina,disciplina,fisica-i,Undefined,/disciplina/fisica-i?type=6&materialid=5191228,2017-11-16 02:10:46,Chrome 61,Digital Net Ltda,None,...,Page View,8,,Windows 7,None,af9ab1ed319a1a8a62dc6e39638c7ee743340eb7,ca122c896b84fa32f750f95c9c163d3e1146c63e24f2dd...,e,known,eacbfbb3d1058820a3679db024155efa9e98230b
7,None,disciplina,disciplina,introducao-ao-estudo-de-direito,Undefined,/disciplina/introducao-ao-estudo-de-direito,2017-11-16 02:10:53,Chrome 61,T-net Wireless E Informa#769tica,None,...,Page View,8,,Windows 10,None,74c44cbf080da372d3f63ae61bd67ed006f1a2a6,e1b0981e40fdef0c39b9e1442ca23aa10eceaee7f1013e...,e,known,7314139889cd861e86608797030931d504dda2e7
8,None,busca,None,None,None,/busca,2017-11-16 02:10:54,Mobile Safari 11,Claro Brazil,None,...,Page View,1,11.1.1,iOS,None,0bf547c5ac802a0e0b3e28b8d5e012066fafa9d6,859fa6b0af443a290eb97b19cc3e16eeea37cc60c05a5d...,e,known,01d36048a25a15778ccda7ffa384bd755b73ae61
9,None,home,Undefined,Undefined,Undefined,/,2017-11-16 02:11:03,Edge 14,Quality Telecom Servicos De Comunicacao Ltda Epp,None,...,Page View,2,,Windows 10,None,943ba5d00ab9607eb2979618e11c10ccec1a17dd,344262de5df32d62ccd1592dfd9aa073a584d0fdd1f164...,e,known,c70f886f19d653d2e000bba9e855da69acef19be


In [7]:
df.columns

['last_accessed_url',
 'page_category',
 'page_category_1',
 'page_category_2',
 'page_category_3',
 'page_name',
 'at',
 'browser',
 'carrier',
 'city_name',
 'clv_total',
 'country',
 'custom_1',
 'custom_2',
 'custom_3',
 'custom_4',
 'device_new',
 'first_accessed_page',
 'install_uuid',
 'language',
 'library_ver',
 'marketing_campaign',
 'marketing_medium',
 'marketing_source',
 'model',
 'name',
 'nth',
 'os_ver',
 'platform',
 'region',
 'session_uuid',
 'studentid_clienttype',
 'type',
 'user_type',
 'uuid']

#### Understanding cities where Passei Direto is more popular

In [8]:
df.groupby('city_name') \
    .count() \
    .filter("`count` >= 100") \
    .sort(col("count").desc()) \
    .collect()

[Row(city_name=None, count=688435),
 Row(city_name='Brasilia', count=1199),
 Row(city_name='Goiânia', count=838),
 Row(city_name='Sao José Dos Pinhais', count=620),
 Row(city_name='Chía', count=583),
 Row(city_name='Porto Alegre', count=524),
 Row(city_name='Campo Grande', count=429),
 Row(city_name='Cuiabá', count=395),
 Row(city_name='Nova Friburgo', count=383),
 Row(city_name='Maringá', count=355),
 Row(city_name='Porto Velho', count=315),
 Row(city_name='Petrópolis', count=280),
 Row(city_name='Joinville', count=238),
 Row(city_name='Palhoça', count=222),
 Row(city_name='Chapecó', count=164),
 Row(city_name='Lisbon', count=158),
 Row(city_name='Passo Fundo', count=158),
 Row(city_name='Vitória', count=157),
 Row(city_name='Toledo', count=148),
 Row(city_name='Criciúma', count=146),
 Row(city_name='Santa Maria', count=145),
 Row(city_name='Pelotas', count=130),
 Row(city_name='Porto', count=123),
 Row(city_name='Santa Cruz Do Sul', count=121),
 Row(city_name='Rio Branco', count=114)

#### Percentage of Mobile users

In [9]:
df.filter(df.browser.contains('Mobile')).count() / df.where(col("browser").isNotNull()).count() * 100

10.192405092268029

#### Most Visited Subjects

In [24]:
df.filter(df.page_category == 'disciplina') \
    .groupby('page_category_2') \
    .count() \
    .filter(df.page_category_2 != 'seguindo') \
    .filter("`count` >= 1400") \
    .sort(col("count").desc()) \
    .collect()

[Row(page_category_2='matematica-financeira', count=2629),
 Row(page_category_2='direito-penal', count=2245),
 Row(page_category_2='administracao', count=1980),
 Row(page_category_2='ciencia-politica', count=1963),
 Row(page_category_2='gestao-da-qualidade', count=1912),
 Row(page_category_2='bioquimica', count=1830),
 Row(page_category_2='metodologia-cientifica', count=1828),
 Row(page_category_2='direito-civil', count=1765),
 Row(page_category_2='direito-penal-iii', count=1757),
 Row(page_category_2='pedagogia', count=1757),
 Row(page_category_2='direito-penal-i', count=1711),
 Row(page_category_2='fenomenos-dos-transportes', count=1672),
 Row(page_category_2='direito', count=1635),
 Row(page_category_2='direito-penal-ii', count=1599),
 Row(page_category_2='fisiologia-humana', count=1556),
 Row(page_category_2='introducao-a-administracao', count=1530),
 Row(page_category_2='contabilidade---ciencias-contabeis', count=1515),
 Row(page_category_2='direito-constitucional-ii', count=1445)

#### Trying to collect student_id

In [38]:
df.select(df.studentid_clienttype).limit(1).collect()

[Row(studentid_clienttype='34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364e097371db586d4ea48@Website')]

In [58]:
# Removing @Website from the end of the string
df.select(df.studentid_clienttype[0:64]).limit(1).collect()

[Row(substring(studentid_clienttype, 0, 64)='34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364e097371db586d4ea48')]

#### Creating another column for student_id

In [43]:
from pyspark.sql.functions import substring

df = df.withColumn("student_id", substring(df.studentid_clienttype, 0, 64))

### Bringing student data from BASE A

In [59]:
students = spark.read.json('BASE A/students.json')
students = students.toDF(*(c.lower() for c in students.columns))

In [60]:
df_students = df.join(students, df.student_id == students.id)

##### Percentage of valid students found inside page_view data

In [64]:
df_students.count() / df.count() * 100

2.7401415281954726

In [65]:
df_students.columns

['last_accessed_url',
 'page_category',
 'page_category_1',
 'page_category_2',
 'page_category_3',
 'page_name',
 'at',
 'browser',
 'carrier',
 'city_name',
 'clv_total',
 'country',
 'custom_1',
 'custom_2',
 'custom_3',
 'custom_4',
 'device_new',
 'first_accessed_page',
 'install_uuid',
 'language',
 'library_ver',
 'marketing_campaign',
 'marketing_medium',
 'marketing_source',
 'model',
 'name',
 'nth',
 'os_ver',
 'platform',
 'region',
 'session_uuid',
 'studentid_clienttype',
 'type',
 'user_type',
 'uuid',
 'student_id',
 'city',
 'courseid',
 'id',
 'registereddate',
 'signupsource',
 'state',
 'studentclient',
 'universityid']

#### Understanding universities where Passei Direto is most popular

In [78]:
# Bring in Universities data
universities = spark.read.json('BASE A/universities.json')
universities = universities.toDF(*(c.lower() for c in universities.columns))
universities = universities.withColumnRenamed("name", "university_name")
# Join Universities data with page_view and students data
df_students_universities = df_students.join(universities, df_students.universityid == universities.id)

In [80]:
df_students_universities.groupby('university_name') \
    .count() \
    .filter("`count` >= 100") \
    .sort(col("count").desc()) \
    .collect()

[Row(university_name='ESTÁCIO', count=7115),
 Row(university_name='UNIP', count=1524),
 Row(university_name='UNINTER', count=1385),
 Row(university_name='ESTÁCIO EAD', count=1136),
 Row(university_name='UNOPAR', count=372),
 Row(university_name='Anhanguera', count=202),
 Row(university_name='UNIDERP - ANHANGUERA', count=182),
 Row(university_name='ANHANGUERA', count=166),
 Row(university_name='FIC', count=133),
 Row(university_name='UFAM', count=130),
 Row(university_name='UFSC', count=106),
 Row(university_name='UNIASSELVI', count=100)]